In [ ]:
!pip install transformers
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, AdamW,RobertaModel,RobertaTokenizer
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
from sklearn.model_selection import train_test_split
import copy
import warnings
from sklearn.metrics import accuracy_score as acc
import torch.optim as optim


warnings.filterwarnings("ignore")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

tokenizer = RobertaTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00


In [ ]:
sentiments=pd.read_csv("/content/drive/MyDrive/transformer/s.csv")

In [ ]:
sentiments=sentiments[["others","joy","sadness","anger","surprise","disgust","fear"]]

In [ ]:
len(sentiments)

2671

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/transformer/train.csv")
inputs = data["tweet"]
labels = data["humor"]



train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    inputs, 
    labels, 
    test_size=0.2, 
    stratify=labels
)


In [ ]:
class createDataset(Dataset):

    def __init__(self, texts, targets, tokenizer, max_len):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
  
    def __len__(self):
        return len(self.texts)
  
    def __getitem__(self, item):
        text = str(self.texts[item])
        target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        input=encoding['input_ids'].flatten()
        

        return {
            'text': text,
            'input_ids': input,
            
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

def create_data_loader(texts, labels, tokenizer, max_len, batch_size):
    
    ds = createDataset(
        texts=texts.to_numpy(),
        targets=labels.to_numpy(),
        
        tokenizer=tokenizer,
        max_len=max_len
  )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
      )

In [ ]:
class Model(nn.Module):
    def __init__(self, latent_dims,max_len,nhid):
        super(Model, self).__init__()

        
        self.lstm = nn.LSTM(input_size=1,num_layers=1,hidden_size=128,dropout=0.5,batch_first=True)
        self.linear = nn.Linear(in_features=nhid*max_len, out_features=1028)
        self.r = nn.ReLU()
        self.t = nn.Tanh()
        self.l = nn.Linear(in_features=1028,out_features=256)
        self.l2=nn.Linear(in_features=256,out_features=2)

        self.s = nn.Softmax(dim=1)

        self.latent_dims=latent_dims
        self.nhid=nhid



        
        

    def forward(self, input_id,bs,ls):

      input_id = input_id.to(torch.float32)
      input_id=input_id.reshape(bs,ls,1)
      out,(h,c)=self.lstm(input_id)
      out=out.flatten(-2)
      out=self.t(out)
      


      decoded = self.linear(out)

      decoded=self.r(decoded)

      decoded=self.l(decoded)

      decoded=self.r(decoded)


      decoded=self.l2(decoded)


      

      decoded=self.s(decoded)

      
      
      

     
      
      return decoded



    def init_hidden(self, bsz):
        weight = next(self.parameters())
        return (weight.new_zeros(self.latent_dims, bsz, self.nhid),
                weight.new_zeros(self.latent_dims, bsz, self.nhid))
      



In [ ]:
batch_size = 64
learning_rate = 0.00001
criterion = nn.CrossEntropyLoss().to(device)
criterion.requires_grad=True
epochs = 30
latentdims=2
nhid=128
max_len=60


In [ ]:
model= Model(latentdims,max_len,nhid)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
train_data_loader = create_data_loader(train_inputs, train_labels,tokenizer, max_len, batch_size)

test_data_loader = create_data_loader(test_inputs, test_labels, tokenizer, max_len, batch_size)

In [13]:
from sklearn.metrics import f1_score as f1

In [14]:
def train_an_epoch(
    model, 
    train_data_loader,
    dev_data_loader,
    criterion, 
    optimizer
):

    

    # These are the metrics that will indicate us how well it's doing the model...
    running_loss = 0
    training_acc=[]
    f1_training=[]
    steps = 0;
    
    for batch in train_data_loader:
        
        b=len(batch["input_ids"])
        # Clean gradients...
        optimizer.zero_grad()
    
        # Get the information from the tokenization... (using GPU)
        input_ids = batch["input_ids"].to(device)
        targets = batch["targets"].to(device)

        # get the model's predictions...
        outputs = model(
            input_id=input_ids,
            bs=b,
            ls=60  
        )

        # Apply the loss function and the perform backward propagation...
       
        loss = criterion(outputs, targets)
       
        loss.backward() 
        optimizer.step()
        
        # update the metrics...

        pred = []
        real=[]
        for output in outputs:
          zero=output[0].item()
          one=output[1].item()
          if zero > one:
            pred.append(0)
          else:
            pred.append(1)

        for t in targets:

          real.append(t.item())

        bacc=acc(real,pred)
        bf1= f1(real,pred)
        running_loss+=loss.item()
        training_acc.append(bacc) 
        f1_training.append(bf1)

        steps+=1
            
    # get the mean of the metrics...
    
    loss = running_loss/steps;
    t_acc=sum(training_acc)/len(training_acc)
    t_f1=sum(f1_training)/len(f1_training)
    
    #acc = running_accs/steps;
    
    # evaluate the model with the validation data set 
    # ("turn off" gradients...)
    with torch.no_grad():
        
        # These are the metrics that will indicate us how well it's doing the model...
        test_acc=[];
        steps_val=0;
        f1_test=[]
        
        for batch in dev_data_loader:

            b= len(batch["input_ids"])
            
            # Get the information from the tokenization... (using GPU)
            input_ids = batch["input_ids"].to(device)
            targets = batch["targets"].to(device)

            # get the model's predictions...
            outputs = model(
                input_id=input_ids,
                bs=b,
                ls=60
                
            )
            
            pred = []
            real=[]
            for output in outputs:
              zero=output[0].item()
              one=output[1].item()
              if zero > one:
                pred.append(0)
              else:
                pred.append(1)

            for t in targets:

              real.append(t.item())

            bacc=acc(real,pred)
            bf1= f1(real,pred)
            test_acc.append(bacc)
            f1_test.append(bf1)


        v_acc=sum(test_acc)/len(test_acc)
        v_f1= sum(f1_test)/len(f1_test)
    
    

    return loss,t_acc,v_acc,t_f1,v_f1

def train_the_model(epochs):
    
    for e in range(epochs):
      #, acc, val_acc
        
        loss,t_acc,v_acc,t_f1,v_f1 = train_an_epoch(
            model, 
            train_data_loader,
            test_data_loader,
            criterion, 
            optimizer
        )
        
        print('--------EPOCH SUMMARY---------')
        print('Epoch ', e+1, ' training loss: ', loss)
        print('Epoch ', e+1, ' training acc: ', t_acc*100, '%')
        print('Epoch ', e+1, ' val acc: ', v_acc*100, '%')
        print('Epoch ', e+1, ' training f1: ', t_f1*100, '%')
        print('Epoch ', e+1, ' val f1: ', v_f1*100, '%')

In [15]:
train_the_model(epochs)

--------EPOCH SUMMARY---------
Epoch  1  training loss:  0.6537026762962341
Epoch  1  training acc:  67.67769607843137 %
Epoch  1  val acc:  67.93478260869566 %
Epoch  1  training f1:  0.0 %
Epoch  1  val f1:  0.0 %
--------EPOCH SUMMARY---------
Epoch  2  training loss:  0.6316905652775484
Epoch  2  training acc:  67.67769607843137 %
Epoch  2  val acc:  67.93478260869566 %
Epoch  2  training f1:  0.0 %
Epoch  2  val f1:  0.0 %
--------EPOCH SUMMARY---------
Epoch  3  training loss:  0.6252783729749567
Epoch  3  training acc:  67.67769607843137 %
Epoch  3  val acc:  67.93478260869566 %
Epoch  3  training f1:  0.0 %
Epoch  3  val f1:  0.0 %
--------EPOCH SUMMARY---------
Epoch  4  training loss:  0.6200393648708568
Epoch  4  training acc:  67.67769607843137 %
Epoch  4  val acc:  67.93478260869566 %
Epoch  4  training f1:  0.0 %
Epoch  4  val f1:  0.0 %
--------EPOCH SUMMARY---------
Epoch  5  training loss:  0.6156783331842983
Epoch  5  training acc:  67.67769607843137 %
Epoch  5  val a